# Extract BICEP-Keck instrument parameters

In [28]:
ipac_warning = [
    "Text file in IPAC table format, read with astropy",
    "from astropy.table import QTable",
    "QTable.read('filename.tbl', format='ascii.ipac')",
    f"Instrument model exported from the Planck NPIPE instrument models",
]

In [21]:
import numpy as np
from astropy.io import fits

from astropy.table import QTable
from pysm3 import units as u
from pathlib import Path
import healpy as hp



In [5]:
import matplotlib.pyplot as plt

%matplotlib inline

In [6]:
for fname in [
    "BK18_B95_bandpass_20210607.txt",
    "BK18_K95_bandpass_20210607.txt",
    "BK18_150_bandpass_20210607.txt",
    "BK18_220_bandpass_20210607.txt",
]:
    file_path = Path(fname)
    if not file_path.exists():
        !wget http://bicepkeck.org/BK18_datarelease/{fname}

--2025-04-23 14:58:33--  http://bicepkeck.org/BK18_datarelease/BK18_B95_bandpass_20210607.txt
Resolving bicepkeck.org (bicepkeck.org)... 140.247.151.131
Connecting to bicepkeck.org (bicepkeck.org)|140.247.151.131|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5272 (5.1K) [text/plain]
Saving to: ‘BK18_B95_bandpass_20210607.txt’

BK18_B95_bandpass_2 100%[===================>]   5.15K  --.-KB/s    in 0s      

2025-04-23 14:58:33 (707 MB/s) - ‘BK18_B95_bandpass_20210607.txt’ saved [5272/5272]

--2025-04-23 14:58:33--  http://bicepkeck.org/BK18_datarelease/BK18_K95_bandpass_20210607.txt
Resolving bicepkeck.org (bicepkeck.org)... 140.247.151.131
Connecting to bicepkeck.org (bicepkeck.org)|140.247.151.131|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6782 (6.6K) [text/plain]
Saving to: ‘BK18_K95_bandpass_20210607.txt’

BK18_K95_bandpass_2 100%[===================>]   6.62K  --.-KB/s    in 0s      

2025-04-23 14:58:33 (1.06 GB/s) - ‘BK18_K

--2025-04-23 14:58:34--  http://bicepkeck.org/BK18_datarelease/BK18_150_bandpass_20210607.txt
Resolving bicepkeck.org (bicepkeck.org)... 140.247.151.131
Connecting to bicepkeck.org (bicepkeck.org)|140.247.151.131|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9524 (9.3K) [text/plain]
Saving to: ‘BK18_150_bandpass_20210607.txt’

BK18_150_bandpass_2 100%[===================>]   9.30K  --.-KB/s    in 0s      

2025-04-23 14:58:34 (44.0 MB/s) - ‘BK18_150_bandpass_20210607.txt’ saved [9524/9524]

--2025-04-23 14:58:34--  http://bicepkeck.org/BK18_datarelease/BK18_220_bandpass_20210607.txt
Resolving bicepkeck.org (bicepkeck.org)... 140.247.151.131
Connecting to bicepkeck.org (bicepkeck.org)|140.247.151.131|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9159 (8.9K) [text/plain]
Saving to: ‘BK18_220_bandpass_20210607.txt’

BK18_220_bandpass_2 100%[===================>]   8.94K  26.4KB/s    in 0.3s    

2025-04-23 14:58:34 (26.4 KB/s) - ‘BK18_

In [10]:
channels = [fname.split('_')[1] for fname in [
    "BK18_B95_bandpass_20210607.txt",
    "BK18_K95_bandpass_20210607.txt",
    "BK18_150_bandpass_20210607.txt",
    "BK18_220_bandpass_20210607.txt",
]]

In [11]:
channels

['B95', 'K95', '150', '220']

In [12]:
bandpasses = {}
for fname, channel in zip([
    "BK18_B95_bandpass_20210607.txt",
    "BK18_K95_bandpass_20210607.txt",
    "BK18_150_bandpass_20210607.txt",
    "BK18_220_bandpass_20210607.txt",
], channels):
    df = pd.read_csv(fname, delim_whitespace=True, comment='#', header=None, names=['freq', 'power', 'RJ'])
    bandpasses[channel] = df

/tmp/ipykernel_339090/1939072109.py:8: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(fname, delim_whitespace=True, comment='#', header=None, names=['freq', 'power', 'RJ'])
/tmp/ipykernel_339090/1939072109.py:8: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(fname, delim_whitespace=True, comment='#', header=None, names=['freq', 'power', 'RJ'])
/tmp/ipykernel_339090/1939072109.py:8: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(fname, delim_whitespace=True, comment='#', header=None, names=['freq', 'power', 'RJ'])
/tmp/ipykernel_339090/1939072109.py:8: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a 

In [13]:
from collections import OrderedDict
from astropy import units as u
from astropy.table import QTable

In [14]:
table = QTable(
    names=[
        "telescope",
        "band",
        "center_frequency",
        "fwhm",
        "nside",
        "bandpass_file",
        "beam_file",
    ],
    dtype=[str, str, float, float, int, str, str],
    units=[None, None, u.GHz, u.arcmin, None, None, None],
)

In [18]:
fwhms = {
    "B95": 24 * u.arcmin,   # 95 GHz (BICEP)
    "K95": 43 * u.arcmin,   # 95 GHz (Keck)
    "150": 30 * u.arcmin,   # 150 GHz
    "220": 20 * u.arcmin    # 220 GHz
}

In [25]:
nside = 1024
hp.nside2resol(nside, arcmin=True)

3.435486411817406

In [26]:
for label in channels:
        bp = bandpasses[label]
        center_frequency = float(''.join(filter(str.isdigit, label))) * u.GHz
        table.add_row(
            OrderedDict(
                telescope="BK18",
                band=label,
                center_frequency=center_frequency,
                fwhm=fwhms[label],
                nside=nside,
                bandpass_file="bandpass_" + label + ".tbl",
            )
        )

In [10]:
table

telescope,band,center_frequency,fwhm,nside,bandpass_file,beam_file
,,GHz,arcmin,,,
str3,str3,float64,float64,int64,str16,str35
LFI,030,30.0,33.102652125,1024,bandpass_030.tbl,Bl_TEB_npipe6v20_030GHzx030GHz.fits
LFI,044,44.0,27.94348615,1024,bandpass_044.tbl,Bl_TEB_npipe6v20_044GHzx044GHz.fits
LFI,070,70.0,13.076459609999999,1024,bandpass_070.tbl,Bl_TEB_npipe6v20_070GHzx070GHz.fits
HFI,100,100.0,9.88,2048,bandpass_100.tbl,Bl_TEB_npipe6v20_100GHzx100GHz.fits
HFI,143,143.0,7.18,2048,bandpass_143.tbl,Bl_TEB_npipe6v20_143GHzx143GHz.fits
HFI,217,217.0,4.87,2048,bandpass_217.tbl,Bl_TEB_npipe6v20_217GHzx217GHz.fits
HFI,353,353.0,4.65,2048,bandpass_353.tbl,Bl_TEB_npipe6v20_353GHzx353GHz.fits
HFI,545,545.0,4.72,2048,bandpass_545.tbl,Bl_npipe6v20_545GHzx545GHz.fits


In [11]:
table.meta["comments"] = ipac_warning
table.write(f"instrument_model.tbl", format="ascii.ipac", overwrite=True)

## Create bandpass files

In [29]:
for label, df in bandpasses.items():
    qtable = QTable(
        names=["bandpass_frequency", "bandpass_weight"],
        units=[u.GHz, None],
        data=[df["freq"].values * u.GHz, df["power"].values],
    )
    qtable.meta["comments"] = ipac_warning
    qtable.write(f"bandpass_{label}.tbl", format="ascii.ipac", overwrite=True)